# Final project of Mobile Robotique 

Student group number 19 : <br/>
Kyan Achtari <br/>
Louis Gavignet <br/>
Louise Genoud <br/>
Joaquim Silveira <br/>

# Introduction
<br/>
The project of Mobile Robotique aims to combine vision, path planning, global navigation, local navigation, and filtering with a kalman filter to move Thymio to the goal.
<br/><br/>
We first create a map composed of differents elements : black obstacles where the Thymio must never go, a blue triangle on Thymio to get his position and orientation in real time, a red square goal, and white small obstacles invisible to the camera.  The camera, using the vision modulus, gets the coordinates of every elements on the map from which we extracts a path. We then use the path planning with Dijkstra to extract the best suited path. 
<br/><br/>
While following the path, Thymio may enconters obstacles not detected by the camera. That's why it constantly scans the near environment with the IR sensors, when one is triggered Thymio avoids the obstacles using local navigation.
<br/><br/>
The robot also use a filter to estimate the position with the help of sensors (camera and motor speed) and to correct it. We choose to implement the kalman filter, which shows a strong efficiency even when the camera is blocked.

# Implementaion with vision

### Software and hardware setup

### Global map setup and construction via image processing

# Path planning

# Motion control

### Global navigation

##### Implementation of the mouvement
We create a fonction called pathing() that uses all the functions needed for mouving Thymio. In every iteration, pathing calls the camera, kalman, and refers to the path given by the path planning. Thymio mouves continiously and actuate the speed of his motors thanks to a proportionnal controller.

##### Rotation 
When Thymio arrives to an intermediate goal, he has to change his orientation to go to the next one. The function get_angle_between() gives the difference between the angles

Then, we make sure the angle belongs to the interval -pi, pi :

##### Controller

We use a proportionnal controller to set the speed of the motors. It takes into account the angle for the correcting or new orientation. We also set a angle minimum under which the robot won't turn, the kalman correction can be very precice so we don't take it in consideration every time. This angle minimum equals 0.4 radians, corresponding to 23 degrees. 

### Local navigation

We first read the 7 horizontal sensor values, stored in "prox.vals", which we multiply by IR.SCALE.DOWN (which is 0.05), if any of them is non-zero, we enter obstacle avoidance:

Then we multiply the sensor vector by LEFT.IR.WEIGHT and RIGHT.IR.WEIGHT for each wheel:

$$
LEFT.IR.WEIGHT = - RIGHT.IR.WEIGHT =
\left(\begin{array}{cc} 
1 \\
2 \\
5 \\
-2 \\
-1 \\
0 \\
0 \\
\end{array}\right)
$$

The speed of each wheel is then their respective weight multiplied by the sensor values, and + 10 :

# Filtering with Kalman filter 

### Overview and prediction

To calculate the beliefs of the robot position we setup a Bayes filter. It's a recrsive filter taht calculte first a prediction of the robot positioon and then update the measurement. We implemented an extended Kalman filter because we conside a nonlinear gaussian state space model, our state are $X = (x,y, \theta)$. The states are modified by the speed of the left and right motors U = (speed_left, spped_right). The nonlinear modification of each paramters can be modelled like this : 
$$ x_{t+1} = x_{t} + v_{average}*dt*cos(\theta) $$ 
$$ y_{t+1} = y_{t} + v_{average}*dt*sin(\theta) $$ 
$$ \theta_{t+1} = \theta_{t} + dt*\omega $$  

<c/><c/>
We can write the systeme above with some matrices :
$$ X_{t+1} = A*X_{t} + B*U $$

<c/><c/>
With the following matrices A and B :
$$
A = 
\left(\begin{array}{cc} 
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 \\
\end{array}\right)
$$

$$
B = 
\left(\begin{array}{cc} 
0.5*dt*cos(\theta) & 0.5*dt*cos(\theta)\\
0.5*dt*sin(\theta) & 0.5*dt*sin(\theta) \\
-dt/wheeldist & dt/wheeeldist \\
\end{array}\right)
$$


The state transition covariance matrix Q, contains the variance of the motors speed :

$$ 
Q = 
\left(\begin{array}{cc}
speedvar & 0\\
0 & speedvar\\
\end{array}\right)
$$

<br/>
The covariance of the obervation noise matrix R, errors come from the camera when measuring a known object :

$$ 
R = 
\left(\begin{array}{cc}
camvar & 0 & 0\\
0 & camvar & 0\\
0 & 0 & camanglevar\\
\end{array}\right)
$$

### Prediction

We can predict the next position of the robot thanks to the set of equations of the nonlinear $X_{t+1}$. We calculate the predication every time interval dt, this periode corresponds to actuation time of our robot. Every dt we calculate the new orientation to follow the path, we update the speed of the motors, and we check with the Kalman filter that Thymio is on the right path.

We also compute the prediction of an estimate of the covariance in this function :
$P = P+B*Q*B^T$

### Update 

Update the Kalman filter state estimate previously with the Kalman gain K, and the difference between the actual state z and the predicted one x. In the mean time, we update the values of P.

$$I = z-X$$
and $$K = P*H^T*S$$  with $$S=R+H*P^T*H$$
then $$X=X+K*I$$
and $$P = P-K*H*P$$

### Filter

We then implement the filter function by calling the prediction function and the update function in this order every dt